<a href="https://colab.research.google.com/github/alokssingh/AI-and-Sustainability/blob/main/information_extraction_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Simple RAG example with Langchain, Ollama and and open-source LLM model

In this example, we first connect to an LLM locally and make request to the LLM that Ollama is serving using LangChain. After that, we generate our RAG application from a PDF file and extract details from that document.

<p align="center">
  <img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2023/07/langchain3.png" alt="Langchain Logo" width="20%">
  <img src="https://bookface-images.s3.amazonaws.com/logos/ee60f430e8cb6ae769306860a9c03b2672e0eaf2.png" alt="Ollama Logo" width="20%">
</p>

Sources:

* https://github.com/svpino/llm
* https://github.com/AIAnytime/Gemma-7B-RAG-using-Ollama/blob/main/Ollama%20Gemma.ipynb
* https://www.youtube.com/watch?v=-MexTC18h20&ab_channel=AIAnytime
* https://www.youtube.com/watch?v=HRvyei7vFSM&ab_channel=Underfitted


# Requirements

* Ollama installed locally

# Install the requirements

If an error is raised related to docarray, refer to this solution: https://stackoverflow.com/questions/76880224/error-using-using-docarrayinmemorysearch-in-langchain-could-not-import-docarray

In [8]:
!pip3 install langchain
!pip3 install langchain_pinecone
!pip3 install langchain[docarray]
!pip3 install docarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 10.1 MB/s eta 0:00:00


In [30]:
!pip install -U pymupdf4llm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.4 MB/s eta 0:00:00


In [ ]:

!pip install langchain-community
!pip install langchain-text-splitters


In [17]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.5 which is incompatible.


In [44]:
!pip install chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.5 MB/s 

In [8]:
cd /content/drive/MyDrive/AI and Sus Oxford

/content/drive/MyDrive/AI and Sus Oxford


# Select the LLM model to use

The model must be downloaded locally to be used, so if you want to run llama3, you should run:

```

ollama pull llama3

```

Check the list of models available for Ollama here: https://ollama.com/library

In [9]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.4 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,153 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,491 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restri

In [1]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

In [46]:
#!ollama pull Phi3
!ollama pull llama3

In [47]:
!ollama list

NAME             ID              SIZE      MODIFIED          
llama3:latest    365c0bd3c000    4.7 GB    2 seconds ago        
Phi3:latest      4f2222927938    2.2 GB    About an hour ago    


In [94]:
#MODEL = "gpt-3.5-turbo"
#MODEL = "mixtral:8x7b"
#MODEL = "gemma:7b"
MODEL = "llama3:latest"
#MODEL = "Phi3:latest" # https://ollama.com/library/llama3

# We instanciate the LLM model and the Embedding model

In [49]:
import langchain
print(langchain.__version__)


1.1.0


In [95]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Give me an inspirational quote")

'Here\'s one:\n\n"Believe you can and you\'re halfway there." - Theodore Roosevelt'

In [51]:
model.invoke("Waht is 2+2?")

'The answer to 2+2 is... 4!'

## Using a parser provided by LangChain, we can transform the LLM output to something more suitable to be read

In [52]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
response_from_model = model.invoke("Give me an inspirational quote")
parsed_response = parser.parse(response_from_model)
print(parsed_response)

Here's one:

"Believe you can and you're halfway there." - Theodore Roosevelt


# We generate the template for the conversation with the instruct-based LLM

We can create a template to structure the conversation effectively.

This template allows us to provide some general context to the Language Learning Model (LLM), which will be utilized for every prompt. This ensures that the model has a consistent background understanding for all interactions.

Additionally, we can include specific context relevant to the particular prompt. This helps the model understand the immediate scenario or topic before addressing the actual question. Following this specific context, we then present the actual question we want the model to answer.

By using this approach, we enhance the model's ability to generate accurate and relevant responses based on both the general and specific contexts provided.

In [53]:
#from langchain.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate

template ="""
Use the following context to answer the question. Be concise and precise.
Extract the answer to the following question from the provided text. Ensure the response is concise, accurate, and specific to the details mentioned in the text. Guidelines:
- Provide only the answer. Do not include the question in the answer. If the information is not explicitly mentioned in the text, respond with: "not available". Avoid adding context, explanations, or assumptions.
- Keep the answer brief and exact.

Context: {context}
Question: {question}
Answer:
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nUse the following context to answer the question. Be concise and precise.\nExtract the answer to the following question from the provided text. Ensure the response is concise, accurate, and specific to the details mentioned in the text. Guidelines:\n- Provide only the answer. Do not include the question in the answer. If the information is not explicitly mentioned in the text, respond with: "not available". Avoid adding context, explanations, or assumptions.\n- Keep the answer brief and exact.\n\nContext: Here is some context\nQuestion: Here is a question\nAnswer:\n'

The model can answer prompts based on the context:

In [54]:
formatted_prompt = prompt.format(context="My parents named me Sergio", question="What's your name?")
response_from_model = model.invoke(formatted_prompt)
parsed_response = parser.parse(response_from_model)
print(parsed_response)

Sergio


But it can't answer what is not provided as context:

In [55]:
formatted_prompt = prompt.format(context="My parents named me Sergio", question="What's my age?")
response_from_model = model.invoke(formatted_prompt)
parsed_response = parser.parse(response_from_model)
print(parsed_response)

Not available


Even previously known info!

In [56]:
formatted_prompt = prompt.format(context="My parents named me Sergio", question="What is 2+2?")
response_from_model = model.invoke(formatted_prompt)
parsed_response = parser.parse(response_from_model)
print(parsed_response)

Not available


# Load an example PDF to do Retrieval Augmented Generation (RAG)

For the example, you can select your own PDF.

In [80]:
import pymupdf4llm

# 1. Convert entire PDF to markdown
#md = pymupdf4llm.to_markdown("/content/drive/MyDrive/AI and Sus Oxford/ibx_ld5_en.pdf")

# 2. (Optional) Split into pages
data = pymupdf4llm.to_markdown("/content/drive/MyDrive/AI and Sus Oxford/ibx_ld5_en.pdf", page_chunks=True)
# data is a list of dicts: one dict per page with markdown text + metadata


In [81]:
data

[{'metadata': {'format': 'PDF 1.7',
   'title': '',
   'author': 'Harleen dhadwal',
   'subject': '',
   'keywords': '',
   'creator': 'Microsoft® Word for Microsoft 365',
   'producer': 'Microsoft® Word for Microsoft 365',
   'creationDate': "D:20231130175353+00'00'",
   'modDate': "D:20231130175353+00'00'",
   'trapped': '',
   'encryption': None,
   'file_path': '/content/drive/MyDrive/AI and Sus Oxford/ibx_ld5_en.pdf',
   'page_count': 2,
   'page': 1},
  'toc_items': [],
  'tables': [],
  'images': [{'number': 18,
    'bbox': Rect(322.79998779296875, 274.39996337890625, 545.5, 497.0999755859375),
    'transform': (222.6999969482422,
     0.0,
     -0.0,
     222.6999969482422,
     322.79998779296875,
     274.39996337890625),
    'width': 618,
    'height': 618,
    'colorspace': 3,
    'cs-name': 'DeviceRGB',
    'xres': 96,
    'yres': 96,
    'bpc': 8,
    'size': 342324,
    'has-mask': False}],
  'graphics': [],
  'text': 'Data Center Technical Specifications\n\n# LD5\n\n\nL

## Converting data to Document objects

In [69]:
from langchain_core.documents import Document

# Convert pymupdf4llm output to Documents
pages = []
for page in data:
    pages.append(Document(
        page_content=page["text"],

    ))


In [89]:
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain_core.documents import Document

def split_markdown_documents(data, chunk_size=300, chunk_overlap=30):
    header_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=[("#", "Header 1"), ("##", "Header 2"),
                             ("###", "Header 3"), ("####", "Header 4")],
        strip_headers=False
    )

    char_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )

    final_docs = []

    for i, page in enumerate(data):
        text = page.get("text", "")
        if not text.strip():
            continue

        # Split markdown headers → list of strings
        sections = header_splitter.split_text(text)
        sections_str = []
        for s in sections:
            if isinstance(s, Document):
                sections_str.append(s.page_content)
            else:
                sections_str.append(s)

        # Split each section into character chunks
        for sec_text in sections_str:
            subchunks = char_splitter.split_text(sec_text)
            for chunk in subchunks:
                final_docs.append(
                    Document(page_content=chunk, metadata={"page_index": i})
                )

    return final_docs


In [90]:
text_documents = split_markdown_documents(
    data,
    chunk_size=1000,
    chunk_overlap=20
)


In [91]:
with open("/content/drive/MyDrive/AI and Sus Oxford/output_after_chunking.txt", "w", encoding="utf-8") as f:
    for i, doc in enumerate(text_documents):
        f.write(f"--- Chunk {i+1} ---\n")
        f.write(doc.page_content + "\n\n")  # now contains actual text


# Store the PDF in a vector space.

From Langchain docs:

`Chromadb is a document index provided by Chroma that stores documents in memory. It is a great starting point for small datasets, where you may not want to launch a database server.`

The execution time of the following block depends on the complexity and longitude of the PDF provided. Try to keep it small and simple for the example.

In [97]:
from chromadb import Client
client = Client()
client.delete_collection("my_collection")


In [98]:
from langchain_community.vectorstores import Chroma

MODEL = "llama3:latest"
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

vectorstore = Chroma.from_documents(
    text_documents,
    embedding=embeddings,
    collection_name="my_collection"
)


# Create retriever of vectors that are similar to be used as context

In [99]:
data_center = 'Equinix '
data_provider = 'Equinix'

In [100]:
questions = [ "Is this pdf contains information about the "+ data_center +"  If yes respond with 'Yes' otherwise say 'No' do not provide explination ",
    "Is this pdf contains information about the"+" "+ data_provider +"  If yes respond with 'Yes' otherwise say 'No' do not provide explination ",
     "Is this pdf Sustainability Report/ ESG Report/ Annual Report .  If yes respond with 'Yes (ESG report/ Sustainability Report/ Annual Report)' otherwise say 'No' do not provide explination ",
    "Is this pdf Specification/technical Report.  If yes respond with 'Yes' otherwise say 'No' do not provide explination",
    "Give me only the name(s)  of the datacenter. Provide the name exactly as mentioned in the text. If none are mentioned, respond with 'not available' do not provide explination",
    "Extract and return only the full address of the data center mentioned in the text. Provide the address exactly as stated in the document. If no address is mentioned, respond only with 'not available'. Do not include explanations, additional details, or reformatted text.",
    "Just Give me only the name of the datacenter operator. Provide the operator as mentioned in the text.If none are mentioned, respond with 'not available' do not provide explination",
    "Just Give me only the city name where the "+ data_center +" datacenter located.  If none are mentioned, respond with 'not available' do not provide explination",
        "Give me only the country name where the" + data_center +"  datacenter located.  If none are mentioned, respond with 'not available' do not provide explination",
"Give me only the  operating status of the datacenter. If active respond with 'Active' otherwise 'Inactive'. If none are mentioned, respond with 'not available' do not provide explination",
    "Give me only the year (number only)  when was the datacenter built. Provide the year as mentioned in the text.If none are mentioned, respond with 'not available' do not provide explination",

    "Does the datacenter have SERVICES Full Cabinets, if yes say 'Yes', otherwise say 'No' do not provide explination",
    "Does the datacenter have SERVICES Shared Rackspace , if yes say 'Yes', otherwise say 'No' do not provide explination",
    "IS the  "+ data_center +" is ISO 27001  Certified , if yes say 'Yes', otherwise say 'No' do not provide explination",
    "IS the  "+ data_center +" is ISO 9001  Certified , if yes say 'Yes', otherwise say 'No' do not provide explination",
    "IS the  "+ data_center +" is ISO 14001  Certified , if yes say 'Yes', otherwise say 'No' do not provide explination",
    "IS the  "+ data_center +" is ISO 22301  Certified , if yes say 'Yes', otherwise say 'No' do not provide explination",
    "IS the  "+ data_center +" is ISO 45001  Certified , if yes say 'Yes', otherwise say 'No' do not provide explination",
    "IS the  "+ data_center +" is ISO 50001  Certified , if yes say 'Yes', otherwise say 'No' do not provide explination",
    "IS the  "+ data_center +" is SOC 1 Type I, Type II, or Type 2 security Certified , if yes say 'Yes', otherwise say 'No' do not provide explination",
    "IS the  "+ data_center +" is SOC 2 Type I, Type II, or Type 2 security Certified , if yes say 'Yes', otherwise say 'No' do not provide explination",
    "IS the  "+ data_center +" is SOC 3 Type I, Type II, or Type 2 security Certified , if yes say 'Yes', otherwise say 'No' do not provide explination",
    "IS the  "+ data_center +" is SAS 70  Certified , if yes say 'Yes', otherwise say 'No' do not provide explination",
    "Does the "+ data_center +" have COMPLIANCE DSS PCI Certified or Certifications , if yes say 'Yes', otherwise say 'No' do not provide explanation",

    "Does the "+ data_center +"  have Automatic Firesupression , if yes say 'Yes', otherwise say 'No' do not provide explanation",
    "Does the "+ data_center +"  have SECURITY Fenced Perimeter if yes respond with 'Yes'.only If none are mentioned, respond with 'not available'do not provide explanation  ",
    "Does the "+ data_center +"  have SECURITY CCTV surveillance if yes respond with 'Yes'.only If none are mentioned, respond with 'not available' do not provide explanation ",
    "Does the "+ data_center +"  have SECURITY Onsite Security Staff if yes respond with 'Yes and how many hours such as 24/7'.only If none are mentioned, respond with 'not available' do not provide explination ",
    "Does the "+ data_center +"  have SECURITY Biometric Access Control if yes respond with 'Yes'.only If none are mentioned, respond with 'not available' do not provide explanation ",
    "Does the "+ data_center +"  have SECURITY Card Access Control if yes respond with 'Yes'.only If none are mentioned, respond with 'not available'do not provide explanation  ",
    "Does the "+ data_center +"  have SECURITY Mantrap Entry if yes respond with 'Yes'.only If none are mentioned, respond with 'not available' do not provide explanation ",

   "Give me the number only of Total Critical Power, of "+ data_center +" . If none are mentioned, respond with 'not available'do not provide explanation ",
    "Does the datacenter have POWER UPS Redundancy if yes the give me only number (for example n+1). If none are mentioned, respond with 'not available'do not provide explination  ",
    "Does the datacenter have POWER Cooling Redundancy if yes the give me only number (for example n+1). If none are mentioned, respond with 'not available' do not provide explination ",


    "Give me only the floor type of the facility/building only mentioned in the text. Provide the type only as mentioned in the text (such as Raised Floor,Mixed). If none are mentioned, respond with 'not available' do not provide explination'",
    "Give me only the size of colocation space or technical or IT space of the facility or building mentioned in the text (just value such as 153,000 m sq) .  If none are mentioned, respond with 'not available' do not provide explination",
    "Give me only the size of BUILDING Total Whitespace of mentioned in the text (just size such as 153,000 m sq).  If none are mentioned, respond with 'not available' do not provide explination",

    "Give me only the number of Building Floors/Halls  mentioned in the text.  If none are mentioned, respond with 'not available''",
    "Give me only the BUILDING Max Floor Load mentioned in the text number only for example 10 kN/m2.  If none are mentioned, respond with 'not available' do not provide explination",
    "Extract and return only the size of the Building Floor-to-Ceiling Height as mentioned in the text. Provide the value exactly as stated in the document. If no such information is mentioned, respond only with 'not available'. Do not include explanations, additional details, or reformatted text.",

    "Give me only  of type of facility/building of  "+ data_center +" . for example if it is bare metal, colocation, enterprise, or hyperscaler, as mentioned in the text.If none are mentioned, respond with 'not available' do not provide explination",

    "Give me only the name(s) of the power system is in use by the "+ data_center +"  . Provide the system exactly as mentioned in the text.If none are mentioned, respond with 'not available' do not provide explination",
    "Give me only the renewable energy power generation capacity available on-site. Provide the capacity as mentioned in the text.If none are mentioned, respond with 'not available' do not provide explination",
"Extract and provide only the Power Usage Effectiveness (PUE) value explicitly mentioned for the  data_center . Output only the numeric value (e.g., 1.4). If no PUE is found in the text, respond with 'not available.' Do not generate, assume, or infer values not present in the text.",
    "Extract and list only the cooling techniques mentioned in the text. Provide the cooling techniques exactly as stated in the document. If no cooling techniques are mentioned, respond with 'not available'. Do not include explanations, additional details, or reformat the extracted text.",
    "Give me only number of the WUE or Water Usage Effectiveness (WUE) of the "+ data_center +"  (for example 1.4). If none is  mentioned, respond with 'not available' do not provide explination",
    "Give me only the name(s) of Who are the reported clients of the  "+ data_provider +"  . List the clients exactly as mentioned in the text.If none are mentioned, respond with 'not available' do not provide explination",
    "Does the  "+ data_center +"   have sustainability policies in place. State 'yes' or 'no' based on the text.If none are mentioned, respond with 'not available' do not provide explination",
    "Extract and list all security measures mentioned in the document for  "+ data_center +" . Provide the security measures exactly as stated in the text. If no security measures are mentioned, respond with 'not available'. Do not include explanations, additional details, or reformat the extracted text.",
    "Give me list of all building overview features provide the systems exactly as mentioned in the text.If none are mentioned, respond with 'not available' do not provide explination",
    "Extract and list all power infrastructure details mentioned in the document for  "+ data_center +"  . Provide the power infrastructure details exactly as stated in the text. If no power infrastructure details are mentioned, respond with 'not available'. Do not include explanations, additional details, or reformat the extracted text.",
   "Extract and return only the certifications and awards assigned to the data center as mentioned in the document. List them exactly as they appear in the text. If no certifications or awards are mentioned, respond only with 'not available'. Do not include explanations, interpretations, or repeat previous responses."
]


In [101]:
for question in questions:
    retrieved_docs = vectorstore.similarity_search(question, k=3)
    retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

    formatted_prompt = prompt.format(context=retrieved_context, question=question)

    response_from_model = model.invoke(formatted_prompt)
    parsed_response = parser.parse(response_from_model)

    # Ensure proper line breaks
    formatted_answer = parsed_response.replace("\\n", "\n")  # if escaped newlines
    formatted_answer = "\n".join(line.strip() for line in formatted_answer.splitlines() if line.strip())

    print(f"Question: {question}\n")
    print(f"Answer:\n{formatted_answer}\n")
    print("="*50)  # separator between questions


Question: Is this pdf contains information about the Equinix   If yes respond with 'Yes' otherwise say 'No' do not provide explination 

Answer:
Yes

Question: Is this pdf contains information about the Equinix  If yes respond with 'Yes' otherwise say 'No' do not provide explination 

Answer:
Yes

Question: Is this pdf Sustainability Report/ ESG Report/ Annual Report .  If yes respond with 'Yes (ESG report/ Sustainability Report/ Annual Report)' otherwise say 'No' do not provide explination 

Answer:
No

Question: Is this pdf Specification/technical Report.  If yes respond with 'Yes' otherwise say 'No' do not provide explination

Answer:
No

Question: Give me only the name(s)  of the datacenter. Provide the name exactly as mentioned in the text. If none are mentioned, respond with 'not available' do not provide explination

Answer:
London IBX [®] Data Center

Question: Extract and return only the full address of the data center mentioned in the text. Provide the address exactly as stat

# Loop to ask-answer questions continously

In [42]:
while True:
    print("Say 'exit' or 'quit' to exit the loop")
    question = input('User question: ')
    print(f"Question: {question}")
    if question.lower() in ["exit", "quit"]:
        print("Exiting the conversation. Goodbye!")
        break
    formatted_prompt = prompt.format(context=retrieved_context, question=question)
    response_from_model = model.invoke(formatted_prompt)
    parsed_response = parser.parse(response_from_model)
    print(f"Answer: {parsed_response}")
    print()

Say 'exit' or 'quit' to exit the loop
User question:  Extract and list all power infrastructure details mentioned in the document for  Equinix   . Provide the power infrastructure details exactly as stated in the text. If no power infrastructure details are mentioned, respond with 'not available'. Do not include explanations, additional details, or reformat the extracted text.
Question:  Extract and list all power infrastructure details mentioned in the document for  Equinix   . Provide the power infrastructure details exactly as stated in the text. If no power infrastructure details are mentioned, respond with 'not available'. Do not include explanations, additional details, or reformat the extracted text.
Answer: - Square feet of colocation space: over 89,575 square meters
- Access to right places and partners for accelerating advantage (implies access to strategic locations but lacks specific infrastructure detail)
- Unrivaled access links with more than 30 European markets and 80 e